In [138]:
import pandas as pd
import os
import statsmodels.api as sm
from sklearn import linear_model
import numpy as np

### Data Gathering
#### This section of the notebooks reads in the data files and stores them im pandas dataframes.
The dataframes frames in this section all have colums of [1960 - 2019] and rows for each state execpt for industy_gdp_by_state_df which goes from [1997-2020] 

In [145]:
csv_path = os.path.join(os.getcwd(), "data/csv")

In [146]:
#Read in all datasets here 

vehicle_registration_df = pd.read_csv(os.path.join(csv_path, "vehicle_registrations_by_state.csv"))
energy_consumption_per_real_gdp_df = pd.read_csv(os.path.join(csv_path, "energy_consumption_per_real_gdp.csv"))
current_dollar_gdp_df = pd.read_csv(os.path.join(csv_path, "Current_dollar_GDP.csv")) #in millions
total_consuption_df = pd.read_csv(os.path.join(csv_path, "total_consuption.csv")) #in billion Btu
industy_gdp_by_state_df = pd.read_csv(os.path.join(csv_path, "industy_gdp_by_state.csv"))
total_population_df = pd.read_csv(os.path.join(csv_path, "total_population.csv"))
real_gdp_df = pd.read_csv(os.path.join(csv_path, "real_GDP.csv")) #in millions

### Data Cleaning

#### This section of the notebook cleans the data frames. 

We start by dropping all the unneeded columns so that each data frame has the same colums. Then we drop any columns that have missing values. This only leaves us with a few columns, so it might be a better idea in the future to replace empty values with the mean value for that year or something similar. 

This section is only evaluting vehicle registration, and population against energy consuption as a proof of concept. We will need to clean and add all the other dataframes to this model.  

In [147]:
columns_to_drop = ['Unnamed: 91', 'Unnamed: 92', 'Unnamed: 93', 'Unnamed: 94',
       'Unnamed: 95', 'Unnamed: 96', 'Unnamed: 97', 'Unnamed: 98',
       'Unnamed: 99', 'Unnamed: 100','Unnamed: 62', 'Unnamed: 63', 'Unnamed: 64', 'Unnamed: 65',
       'Unnamed: 66', 'Unnamed: 67', 'Unnamed: 68', 'Unnamed: 69',
       'Unnamed: 70', 'Unnamed: 71', 'Unnamed: 72', 'Unnamed: 73',
       'Unnamed: 74', 'Unnamed: 75', 'Unnamed: 76', 'Unnamed: 77',
       'Unnamed: 78', 'Unnamed: 79', 'Unnamed: 80', 'Unnamed: 81',
       'Unnamed: 82', 'Unnamed: 83', 'Unnamed: 84', 'Unnamed: 85',
       'Unnamed: 86', 'Unnamed: 87', 'Unnamed: 88', 'Unnamed: 89',
       'Unnamed: 90', 'Unnamed: 61']

In [148]:
vehicle_registration_df.drop(index = [0,1,2,3,4,5,6,7,8,9,61,62,64,63], inplace = True)
vehicle_registration_df = vehicle_registration_df.dropna(axis='columns')
vehicle_registration_df.head()
vehicle_registration_columns = vehicle_registration_df.columns

In [149]:
total_population_df.drop(index = [51], inplace = True)
total_population_df = total_population_df.drop( columns = columns_to_drop)
total_population_df = total_population_df.dropna(axis='columns')
total_population_columns = total_population_df.columns

In [150]:
total_consuption_df.drop(index = [51], inplace = True)
total_consuption_df = total_consuption_df.drop( columns = columns_to_drop)
total_consuption_df = total_consuption_df.dropna(axis='columns')
total_consuption_df_columns = total_consuption_df.columns

In [111]:
#Use the columns that are in each dataframe after columns with empty values have been dropped. 
columns_to_evaluate = list(set(vehicle_registration_columns).intersection(total_population_columns).intersection(total_consuption_df_columns))

In [114]:
#ensure each column we are going to evaluate has the same number of values 
for col in columns_to_evaluate:
    print("year" , col , "length for vechical registration:" , len(vehicle_registration_df[col]))
    print("year" , col , "length for energy consumption:" , len(total_consuption_df[col]))
    print("year" , col , "length for population:" , len(total_population_df[col]))
    print("")

year 2010 length for vechical registration: 51
year 2010 length for energy consumption: 51
year 2010 length for population: 51

year 2007 length for vechical registration: 51
year 2007 length for energy consumption: 51
year 2007 length for population: 51

year 2018 length for vechical registration: 51
year 2018 length for energy consumption: 51
year 2018 length for population: 51

year 2009 length for vechical registration: 51
year 2009 length for energy consumption: 51
year 2009 length for population: 51

year 2008 length for vechical registration: 51
year 2008 length for energy consumption: 51
year 2008 length for population: 51

year 2019 length for vechical registration: 51
year 2019 length for energy consumption: 51
year 2019 length for population: 51

year 1998 length for vechical registration: 51
year 1998 length for energy consumption: 51
year 1998 length for population: 51



In [153]:
# loop through the data frames and add each value to data_point_pairs array. 
# The data_point_pairs array will be the [vehicle registration, population] value for each year and each state
# The total_consumption_vals will be the cooresponding energy consuption value 
# for the [vehicle registration, population] data point
data_point_pairs = []
total_consumption_vals = []
for col in columns_to_evaluate:
    for i in range(0,51):
        pair = [vehicle_registration_df.iloc[i][col], total_population_df.iloc[i][col]]
        data_point_pairs.append(pair)
        
        total_consumption_vals.append(total_consuption_df.iloc[i][col]) 

In [164]:
print("vehicle registration:" , data_point_pairs[0][0])
print("population: ", data_point_pairs[0][1])
print("total energy consuption:" ,total_consumption_vals[0])

vehicle registration: 4653838.0
population:  714
total energy consuption: 640590


### Data Analysis

#### This section of the notebooks creates a linear regression moel for energy consuption.

Right now, the model is only using population and vehicle registration to predict energy comnsuption. We will need to include the other dataframes.

In the model summary, x1 represents vehicle regisration and x2 represents population. There are some other values in the summary that give us a good indication as to how well our model fits energy consuption such at the r squared value and F statistic.

We will have to look into why the coefficient for x2 is 291. It should be between (-1,1), so something is throwing the model off. 

In [169]:
# A potential library we can use for regression analysis 

X = data_point_pairs
y = total_consumption_vals
lm = linear_model.LinearRegression()
model = lm.fit(X,y)

#predict energy consuption for vehicle registration = 400000 , population =  800 (million)
predictions = lm.predict([[400000, 800]])
print("Predicted energy consumpion for vehicle registration = 400000 , population =  800 (million):", predictions )

model = sm.OLS(y, X).fit()
model.summary()



Predicted energy consumpion for vehicle registration = 400000 , population =  800 (million): [542276.34089566]


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.873
Model:                            OLS   Adj. R-squared (uncentered):              0.872
Method:                 Least Squares   F-statistic:                              1216.
Date:                Sat, 23 Oct 2021   Prob (F-statistic):                   1.38e-159
Time:                        13:19:53   Log-Likelihood:                         -5441.9
No. Observations:                 357   AIC:                                  1.089e+04
Df Residuals:                     355   BIC:                                  1.090e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0006      0.014      0.045      0.964      -0.026       0.027
x2           291.2033     10.900     26.717      0.000     269.767     312.639
==============================================================================
Omnibus:                      196.037   Durbin-Watson:                   1.829
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2908.640
Skew:                           1.942   Prob(JB):                         0.00
Kurtosis:                      16.433   Cond. No.                     1.49e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.49e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""